In [7]:
# STUDENT_NAME = Aakash Khot #Put your name
# STUDENT_ROLLNO = IMT2020512 #Put your roll number
# CODE_COMPLETE = False 
# set the above to True if you were able to complete the code
# and that you feel your model can generate a good result
# otherwise keep it as False
# Don't lie about this. This is so that we don't waste time with
# the autograder and just perform a manual check
# If the flag above is True and your code crashes, that's
# an instant deduction of 2 points on the assignment.

## No code within "PROTECTED" can be modified.
## We expect this part to be VERBATIM.
## IMPORTS 
## No other library imports other than the below are allowed.
## No, not even Scipy
import numpy as np 
import pandas as pd 
import sklearn.model_selection as model_selection 
import sklearn.preprocessing as preprocessing 
import sklearn.metrics as metrics 
from tqdm import tqdm # You can make lovely progress bars using this
from sklearn.model_selection import train_test_split, StratifiedKFold  # new added ...................
## FILE READING: 
## You are not permitted to read any files other than the ones given below.
X_train = pd.read_csv("train_X.csv",index_col=0).to_numpy()
y_train = pd.read_csv("train_y.csv",index_col=0).to_numpy().reshape(-1,)
X_test = pd.read_csv("test_X.csv",index_col=0).to_numpy()
submissions_df = pd.read_csv("sample_submission.csv",index_col=0)

##FILE WRITING:
# You are not permitted to write to any file other than the one given below.
# submissions_df.to_csv("{}__{}.csv".format(STUDENT_ROLLNO,STUDENT_NAME))


In [ ]:
# from google.colab import drive
# drive.mount('https://drive.google.com/drive/folders/1WoZKrEoJVxY6pooXaxh5ofgaverTtNZQ?usp=share_link')

In [8]:
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [9]:
X_train=X_train/255
X_test=X_test/255

In [10]:
rand=np.arange(60000)
np.random.shuffle(rand)
l=45000
train_no=rand[:l]

val_no=np.setdiff1d(rand,train_no)

X_train,X_valid=X_train[train_no,:],X_train[val_no,:]

y_train,y_valid=y_train[train_no],y_train[val_no]

In [20]:
X_train=X_train.T
X_valid=X_valid.T
y_train=y_train.T
y_valid=y_valid.T

In [17]:

def ReLU(x):
    return np.maximum(x, 0)
def diff_ReLU(x):
    return x>0

def Softmax(x):
    return np.exp(x)/sum(np.exp(x))

def Forward_Prop(Weight1, bias1, Weight2, bias2, X):
    Forward1 = Weight1.dot(X) + bias1
    Activation1 = ReLU(Forward1)
    Forward2 = Weight2.dot(Activation1) + bias2
    Activation2 = Softmax(Forward2)
    return Forward1, Activation1, Forward2, Activation2
  
def One_hot_encoding(Y,n):
    Y_new = np.zeros((Y.size, n))
    Y_new[np.arange(Y.size), Y] = 1
    return Y_new.T

def Backward_Prop(Forward1, Activation1, Forward2, Activation2, Weight1, Weight2, X, Y,l,n):
    Y_new = One_hot_encoding(Y,n)
    diff_Forward2 = Activation2 - Y_new
    diff_Weight2 = diff_Forward2.dot(Activation1.T)/l
    diff_bias2 = np.sum(diff_Forward2)/l
    diff_Forward1 = Weight2.T.dot(diff_Forward2)*diff_ReLU(Forward1)
    diff_Weight1 =diff_Forward1.dot(X.T)/l
    diff_bias1 = np.sum(diff_Forward1)/l
    return diff_Weight1, diff_bias1, diff_Weight2, diff_bias2
  
def Update(Weight1, bias1, Weight2, bias2, diff_Weight1, diff_bias1, diff_Weight2, diff_bias2, learning_rate):
    Weight1 = Weight1 - learning_rate*diff_Weight1 
    Weight2 = Weight2 - learning_rate*diff_Weight2 
    bias1 = bias1 - learning_rate*diff_bias1    
    bias2 = bias2 - learning_rate*diff_bias2    
    return Weight1, bias1, Weight2, bias2

def Accuracy(y_pred,y):
  Correct = np.sum(y_pred==y,axis=0)
  return Correct/len(y)

def gradient_descent(Weight1,bias1,Weight2,bias2,X,Y,learning_rate,iterations,l,n):
   
    for i in range(iterations):
        Forward1, Activation1, Forward2, Activation2 = Forward_Prop(Weight1, bias1, Weight2, bias2, X)
        diff_Weight1, diff_bias1, diff_Weight2, diff_bias2 = Backward_Prop(Forward1, Activation1, Forward2, Activation2, Weight1, Weight2, X, Y,l,n)
        Weight1, bias1, Weight2, bias2 = Update(Weight1, bias1, Weight2, bias2, diff_Weight1, diff_bias1, diff_Weight2, diff_bias2, learning_rate)
        if i % 20 == 0:
            print("Iteration: ", i)
            predictions = np.argmax(Activation2, 0)
            print(Accuracy(predictions, Y))
    return Weight1, bias1, Weight2, bias2

In [18]:
n=200
Weight1 = np.random.rand(n, 784) - 0.5
bias1 = np.random.rand(n, 1) - 0.5
Weight2 = np.random.rand(n, n) - 0.5
bias2 = np.random.rand(n, 1) - 0.5
Weight1, bias1, Weight2, bias2 = gradient_descent(Weight1,bias1,Weight2,bias2,X_train, y_train, 0.25, 500,l,n)

Iteration:  0
0.006577777777777778
Iteration:  20
0.7435333333333334
Iteration:  40
0.8172
Iteration:  60
0.8471777777777778
Iteration:  80
0.8659777777777777
Iteration:  100
0.8780444444444444
Iteration:  120
0.8871111111111111
Iteration:  140
0.8934444444444445
Iteration:  160
0.8982
Iteration:  180
0.9030888888888889
Iteration:  200
0.9068
Iteration:  220
0.9107777777777778
Iteration:  240
0.9142888888888889
Iteration:  260
0.9172222222222223
Iteration:  280
0.9198888888888889
Iteration:  300
0.9224666666666667
Iteration:  320
0.9244
Iteration:  340
0.9261111111111111
Iteration:  360
0.9276222222222222
Iteration:  380
0.9290222222222222
Iteration:  400
0.9306666666666666
Iteration:  420
0.9320666666666667
Iteration:  440
0.9338
Iteration:  460
0.9351555555555555
Iteration:  480
0.9362888888888888


In [21]:

Forward1, Activation1, Forward2, Activation2 = Forward_Prop(Weight1, bias1, Weight2, bias2, X_valid)
predictions = np.argmax(Activation2, 0)
print(Accuracy(predictions, y_valid))

0.9284
